# ML101: Classical Machine Learning Algorithms

This notebook provides an interactive exploration of classical machine learning algorithms implemented from scratch.

## Table of Contents

1. [Linear Regression](#linear-regression)
2. [Logistic Regression](#logistic-regression)
3. [K-Nearest Neighbors](#k-nearest-neighbors)
4. [Data Preprocessing](#data-preprocessing)
5. [Model Evaluation](#model-evaluation)
6. [Algorithm Comparison](#algorithm-comparison)

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split

# Import our ML101 implementations
from ml101 import (
    LinearRegression, LogisticRegression, KNearestNeighbors,
    StandardScaler, MinMaxScaler, train_test_split as ml101_train_test_split
)
from ml101.utils.metrics import ClassificationMetrics, RegressionMetrics
from ml101.utils.preprocessing import PolynomialFeatures

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All libraries loaded successfully!")
print(f"📦 ML101 version: {getattr(__import__('ml101'), '__version__', 'unknown')}")

## Linear Regression

Linear regression is a fundamental algorithm for predicting continuous target variables. Let's explore both the Normal Equation and Gradient Descent methods.

In [ ]:
# Generate synthetic linear data
X, y = make_regression(n_samples=200, n_features=1, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Feature dimensions: {X_train.shape[1]}")

In [ ]:
# Compare Normal Equation vs Gradient Descent
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Method 1: Normal Equation
model_normal = LinearRegression(method='normal')
model_normal.fit(X_train, y_train)

# Method 2: Gradient Descent
model_gd = LinearRegression(method='gradient_descent', learning_rate=0.01, max_iterations=1000)
model_gd.fit(X_train, y_train)

# Create prediction line
X_plot = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
y_plot_normal = model_normal.predict(X_plot)
y_plot_gd = model_gd.predict(X_plot)

# Plot 1: Normal Equation
axes[0].scatter(X_train, y_train, alpha=0.6, label='Training Data')
axes[0].scatter(X_test, y_test, alpha=0.6, label='Test Data', color='red')
axes[0].plot(X_plot, y_plot_normal, color='green', linewidth=2, label='Normal Equation')
axes[0].set_title('Linear Regression - Normal Equation')
axes[0].set_xlabel('X')
axes[0].set_ylabel('y')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Gradient Descent
axes[1].scatter(X_train, y_train, alpha=0.6, label='Training Data')
axes[1].scatter(X_test, y_test, alpha=0.6, label='Test Data', color='red')
axes[1].plot(X_plot, y_plot_gd, color='orange', linewidth=2, label='Gradient Descent')
axes[1].set_title('Linear Regression - Gradient Descent')
axes[1].set_xlabel('X')
axes[1].set_ylabel('y')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Cost History
if model_gd.cost_history:
    axes[2].plot(model_gd.cost_history, color='purple')
    axes[2].set_title('Cost Function During Training')
    axes[2].set_xlabel('Iteration')
    axes[2].set_ylabel('Mean Squared Error')
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print results
print("\nResults Comparison:")
print(f"Normal Equation    - Weight: {model_normal.weights[0]:.4f}, Bias: {model_normal.bias:.4f}")
print(f"Gradient Descent   - Weight: {model_gd.weights[0]:.4f}, Bias: {model_gd.bias:.4f}")
print(f"\nR² Scores:")
print(f"Normal Equation    - Train: {model_normal.score(X_train, y_train):.4f}, Test: {model_normal.score(X_test, y_test):.4f}")
print(f"Gradient Descent   - Train: {model_gd.score(X_train, y_train):.4f}, Test: {model_gd.score(X_test, y_test):.4f}")

## Logistic Regression

Logistic regression is used for binary and multiclass classification problems. It uses the sigmoid function to model probabilities.

In [ ]:
# Generate classification data
X_clf, y_clf = make_classification(n_samples=300, n_features=2, n_redundant=0, 
                                  n_informative=2, n_clusters_per_class=1, 
                                  random_state=42)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.3, random_state=42)

# Scale features for better convergence
scaler = StandardScaler()
X_clf_train_scaled = scaler.fit_transform(X_clf_train)
X_clf_test_scaled = scaler.transform(X_clf_test)

print(f"Classification dataset shape: {X_clf.shape}")
print(f"Number of classes: {len(np.unique(y_clf))}")
print(f"Class distribution: {np.bincount(y_clf)}")

In [ ]:
# Train logistic regression
log_model = LogisticRegression(learning_rate=0.1, max_iterations=1000)
log_model.fit(X_clf_train_scaled, y_clf_train)

# Make predictions
y_pred = log_model.predict(X_clf_test_scaled)
y_proba = log_model.predict_proba(X_clf_test_scaled)

# Calculate metrics
accuracy = ClassificationMetrics.accuracy_score(y_clf_test, y_pred)
precision, recall, fscore = ClassificationMetrics.precision_recall_fscore(y_clf_test, y_pred, average='binary')

print(f"\nLogistic Regression Results:")
print(f"Training Accuracy: {log_model.score(X_clf_train_scaled, y_clf_train):.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {fscore:.4f}")

In [ ]:
# Visualize logistic regression results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Decision boundary
def plot_decision_boundary(model, X, y, scaler, ax, title):
    h = 0.1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict_proba(mesh_points)[:, 1]
    Z = Z.reshape(xx.shape)
    
    contour = ax.contourf(xx, yy, Z, levels=50, alpha=0.6, cmap=plt.cm.RdYlBu)
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    ax.set_title(title)
    ax.set_xlabel('Feature 1 (scaled)')
    ax.set_ylabel('Feature 2 (scaled)')
    return contour

contour = plot_decision_boundary(log_model, X_clf_train_scaled, y_clf_train, scaler, axes[0], 
                                'Logistic Regression Decision Boundary')
plt.colorbar(contour, ax=axes[0])

# Plot 2: Cost history
if log_model.cost_history:
    axes[1].plot(log_model.cost_history, color='blue')
    axes[1].set_title('Cost Function History')
    axes[1].set_xlabel('Iteration')
    axes[1].set_ylabel('Logistic Loss')
    axes[1].grid(True, alpha=0.3)

# Plot 3: Confusion Matrix
cm = ClassificationMetrics.confusion_matrix(y_clf_test, y_pred)
im = axes[2].imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[2].figure.colorbar(im, ax=axes[2])
axes[2].set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]),
           xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'],
           title='Confusion Matrix', ylabel='True label', xlabel='Predicted label')

# Add text annotations to confusion matrix
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        axes[2].text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

## K-Nearest Neighbors

KNN is a simple, instance-based learning algorithm that classifies instances based on the majority class of their k nearest neighbors.

In [ ]:
# Generate KNN data
X_knn, y_knn = make_classification(n_samples=300, n_features=2, n_redundant=0, 
                                  n_informative=2, n_clusters_per_class=1, 
                                  random_state=42)
X_knn_train, X_knn_test, y_knn_train, y_knn_test = train_test_split(X_knn, y_knn, test_size=0.3, random_state=42)

print(f"KNN dataset shape: {X_knn.shape}")
print(f"Number of classes: {len(np.unique(y_knn))}")
print(f"Class distribution: {np.bincount(y_knn)}")

In [ ]:
# Analyze effect of different k values
k_values = range(1, 21)
train_scores = []
test_scores = []

for k in k_values:
    knn_model = KNearestNeighbors(k=k)
    knn_model.fit(X_knn_train, y_knn_train)
    
    train_score = knn_model.score(X_knn_train, y_knn_train)
    test_score = knn_model.score(X_knn_test, y_knn_test)
    
    train_scores.append(train_score)
    test_scores.append(test_score)

# Plot k-value analysis
plt.figure(figsize=(10, 6))
plt.plot(k_values, train_scores, 'o-', label='Training Accuracy', color='blue')
plt.plot(k_values, test_scores, 'o-', label='Test Accuracy', color='red')
plt.xlabel('k (Number of Neighbors)')
plt.ylabel('Accuracy')
plt.title('KNN Performance vs k Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Find best k
best_k = k_values[np.argmax(test_scores)]
print(f"Best k value: {best_k} (Test Accuracy: {max(test_scores):.4f})")

In [ ]:
# Visualize KNN decision boundaries for different k values
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

k_demo_values = [1, 3, 5, 10, 15, 25]

def plot_knn_boundary(model, X, y, ax, title):
    h = 0.1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(mesh_points)
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.6, cmap=plt.cm.RdYlBu)
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    ax.set_title(title)
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

for i, k in enumerate(k_demo_values):
    knn_model = KNearestNeighbors(k=k)
    knn_model.fit(X_knn_train, y_knn_train)
    
    test_acc = knn_model.score(X_knn_test, y_knn_test)
    plot_knn_boundary(knn_model, X_knn_train, y_knn_train, axes[i], 
                     f'KNN (k={k})\nAccuracy: {test_acc:.3f}')

plt.tight_layout()
plt.show()

## Data Preprocessing

Data preprocessing is crucial for machine learning. Let's explore different scaling methods and feature engineering techniques.

In [ ]:
# Generate data with different scales
np.random.seed(42)
X_preprocess = np.random.randn(200, 3) * [100, 10, 1] + [500, 50, 5]

# Apply different scaling methods
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

X_standard = standard_scaler.fit_transform(X_preprocess)
X_minmax = minmax_scaler.fit_transform(X_preprocess)

# Visualize scaling effects
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

datasets = [X_preprocess, X_standard, X_minmax]
titles = ['Original Data', 'StandardScaler', 'MinMaxScaler']

for i, (data, title) in enumerate(zip(datasets, titles)):
    bp = axes[i].boxplot(data, labels=['Feature 1', 'Feature 2', 'Feature 3'])
    axes[i].set_title(title)
    axes[i].set_ylabel('Values')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Data Statistics Before and After Scaling:")
print("\nOriginal Data:")
print(f"Mean: {np.mean(X_preprocess, axis=0)}")
print(f"Std:  {np.std(X_preprocess, axis=0)}")

print("\nStandardScaler:")
print(f"Mean: {np.mean(X_standard, axis=0)}")
print(f"Std:  {np.std(X_standard, axis=0)}")

print("\nMinMaxScaler:")
print(f"Min: {np.min(X_minmax, axis=0)}")
print(f"Max: {np.max(X_minmax, axis=0)}")

In [ ]:
# Polynomial features demonstration
# Generate 1D nonlinear data
np.random.seed(42)
X_poly_demo = np.random.uniform(-2, 2, 100).reshape(-1, 1)
y_poly_demo = X_poly_demo.ravel()**3 + 0.5 * X_poly_demo.ravel()**2 + np.random.normal(0, 0.5, 100)

# Apply polynomial features with different degrees
degrees = [1, 2, 3, 4]
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

X_plot = np.linspace(-2, 2, 100).reshape(-1, 1)

for i, degree in enumerate(degrees):
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X_poly_demo)
    X_plot_poly = poly.transform(X_plot)
    
    # Fit linear regression on polynomial features
    theta = np.linalg.pinv(X_poly.T @ X_poly) @ X_poly.T @ y_poly_demo
    y_plot = X_plot_poly @ theta
    
    # Calculate R² score
    y_pred = X_poly @ theta
    r2 = RegressionMetrics.r2_score(y_poly_demo, y_pred)
    
    axes[i].scatter(X_poly_demo, y_poly_demo, alpha=0.6, label='Data')
    axes[i].plot(X_plot, y_plot, color='red', linewidth=2, 
                label=f'Polynomial degree {degree}')
    axes[i].set_title(f'Degree {degree}\nR² = {r2:.3f}')
    axes[i].set_xlabel('X')
    axes[i].set_ylabel('y')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Polynomial Feature Engineering Effects')
plt.tight_layout()
plt.show()

## Model Evaluation

Proper evaluation of machine learning models is crucial. Let's explore different metrics and evaluation techniques.

In [ ]:
# Generate sample data for evaluation demo
X_eval, y_eval = make_classification(n_samples=400, n_features=2, n_redundant=0, 
                                   n_informative=2, n_clusters_per_class=1, 
                                   random_state=42)
X_eval_train, X_eval_test, y_eval_train, y_eval_test = train_test_split(
    X_eval, y_eval, test_size=0.3, random_state=42)

# Scale features
scaler_eval = StandardScaler()
X_eval_train_scaled = scaler_eval.fit_transform(X_eval_train)
X_eval_test_scaled = scaler_eval.transform(X_eval_test)

# Train multiple models for comparison
models = {
    'Logistic Regression': LogisticRegression(learning_rate=0.1, max_iterations=1000),
    'KNN (k=3)': KNearestNeighbors(k=3),
    'KNN (k=7)': KNearestNeighbors(k=7),
    'KNN (k=15)': KNearestNeighbors(k=15),
}

results = {}
predictions = {}

for name, model in models.items():
    if 'Logistic' in name:
        model.fit(X_eval_train_scaled, y_eval_train)
        y_pred = model.predict(X_eval_test_scaled)
        train_acc = model.score(X_eval_train_scaled, y_eval_train)
        test_acc = model.score(X_eval_test_scaled, y_eval_test)
    else:
        model.fit(X_eval_train, y_eval_train)
        y_pred = model.predict(X_eval_test)
        train_acc = model.score(X_eval_train, y_eval_train)
        test_acc = model.score(X_eval_test, y_eval_test)
    
    predictions[name] = y_pred
    results[name] = {'train_acc': train_acc, 'test_acc': test_acc}
    
    # Calculate additional metrics
    precision, recall, fscore = ClassificationMetrics.precision_recall_fscore(
        y_eval_test, y_pred, average='binary')
    results[name].update({'precision': precision, 'recall': recall, 'fscore': fscore})

# Display results
print("Model Comparison Results:")
print("=" * 80)
print(f"{'Model':<20} {'Train Acc':<10} {'Test Acc':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
print("=" * 80)

for name, metrics in results.items():
    print(f"{name:<20} {metrics['train_acc']:<10.3f} {metrics['test_acc']:<10.3f} "
          f"{metrics['precision']:<10.3f} {metrics['recall']:<10.3f} {metrics['fscore']:<10.3f}")

In [ ]:
# Visualize confusion matrices for each model
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, (name, y_pred) in enumerate(predictions.items()):
    cm = ClassificationMetrics.confusion_matrix(y_eval_test, y_pred)
    
    im = axes[i].imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    axes[i].figure.colorbar(im, ax=axes[i])
    axes[i].set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]),
               xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'],
               title=f'{name}\nAccuracy: {results[name]["test_acc"]:.3f}',
               ylabel='True label', xlabel='Predicted label')
    
    # Add text annotations
    thresh = cm.max() / 2.
    for row in range(cm.shape[0]):
        for col in range(cm.shape[1]):
            axes[i].text(col, row, format(cm[row, col], 'd'), ha="center", va="center",
                        color="white" if cm[row, col] > thresh else "black")

plt.tight_layout()
plt.show()

## Algorithm Comparison

Let's compare all our algorithms on the same dataset to understand their strengths and weaknesses.

In [ ]:
# Create a comprehensive comparison
datasets = {
    'Linear Separable': make_classification(n_samples=300, n_features=2, n_redundant=0, 
                                          n_informative=2, n_clusters_per_class=1, 
                                          random_state=42),
    'Non-linear': make_classification(n_samples=300, n_features=2, n_redundant=0, 
                                    n_informative=2, n_clusters_per_class=2, 
                                    random_state=42)
}

algorithms = {
    'Logistic Regression': LogisticRegression(learning_rate=0.1, max_iterations=1000),
    'KNN (k=5)': KNearestNeighbors(k=5),
}

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for dataset_idx, (dataset_name, (X_comp, y_comp)) in enumerate(datasets.items()):
    X_comp_train, X_comp_test, y_comp_train, y_comp_test = train_test_split(
        X_comp, y_comp, test_size=0.3, random_state=42)
    
    # Scale for logistic regression
    scaler_comp = StandardScaler()
    X_comp_train_scaled = scaler_comp.fit_transform(X_comp_train)
    X_comp_test_scaled = scaler_comp.transform(X_comp_test)
    
    for alg_idx, (alg_name, model) in enumerate(algorithms.items()):
        ax = axes[dataset_idx, alg_idx]
        
        if 'Logistic' in alg_name:
            model.fit(X_comp_train_scaled, y_comp_train)
            test_acc = model.score(X_comp_test_scaled, y_comp_test)
            
            # Plot decision boundary
            h = 0.1
            x_min, x_max = X_comp_train_scaled[:, 0].min() - 1, X_comp_train_scaled[:, 0].max() + 1
            y_min, y_max = X_comp_train_scaled[:, 1].min() - 1, X_comp_train_scaled[:, 1].max() + 1
            xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
            
            mesh_points = np.c_[xx.ravel(), yy.ravel()]
            Z = model.predict_proba(mesh_points)[:, 1]
            Z = Z.reshape(xx.shape)
            
            ax.contourf(xx, yy, Z, levels=50, alpha=0.6, cmap=plt.cm.RdYlBu)
            ax.scatter(X_comp_train_scaled[:, 0], X_comp_train_scaled[:, 1], 
                      c=y_comp_train, cmap=plt.cm.RdYlBu, edgecolors='black')
        else:
            model.fit(X_comp_train, y_comp_train)
            test_acc = model.score(X_comp_test, y_comp_test)
            
            # Plot decision boundary
            h = 0.1
            x_min, x_max = X_comp_train[:, 0].min() - 1, X_comp_train[:, 0].max() + 1
            y_min, y_max = X_comp_train[:, 1].min() - 1, X_comp_train[:, 1].max() + 1
            xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
            
            mesh_points = np.c_[xx.ravel(), yy.ravel()]
            Z = model.predict(mesh_points)
            Z = Z.reshape(xx.shape)
            
            ax.contourf(xx, yy, Z, alpha=0.6, cmap=plt.cm.RdYlBu)
            ax.scatter(X_comp_train[:, 0], X_comp_train[:, 1], 
                      c=y_comp_train, cmap=plt.cm.RdYlBu, edgecolors='black')
        
        ax.set_title(f'{alg_name} on {dataset_name}\nAccuracy: {test_acc:.3f}')
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

## Summary and Key Takeaways

### 🎯 What You've Learned

This tutorial covered the core machine learning algorithms available in ML101:

#### Linear Regression
- **Best for**: Continuous target variables with linear relationships
- **Methods**: Normal Equation (exact) and Gradient Descent (scalable)
- **Use cases**: Predicting house prices, stock prices, temperature

#### Logistic Regression
- **Best for**: Binary and multiclass classification
- **Provides**: Probability estimates and interpretable coefficients
- **Assumes**: Linear decision boundary in feature space

#### K-Nearest Neighbors (KNN)
- **Best for**: Non-linear decision boundaries
- **Advantages**: No assumptions about data distribution
- **Considerations**: Sensitive to choice of k and distance metric
- **Computational cost**: High for large datasets (lazy learning)

#### Data Preprocessing
- **Scaling**: Critical for distance-based algorithms and gradient descent
- **Feature engineering**: Polynomial features can help linear models
- **Evaluation**: Multiple metrics provide comprehensive insight

### 📊 Algorithm Comparison

| Algorithm | Linear Data | Non-linear Data | Speed | Interpretability | Memory |
|-----------|-------------|-----------------|-------|------------------|--------|
| Linear Regression | ✅ | ❌ | ✅ | ✅ | ✅ |
| Logistic Regression | ✅ | ❌ | ✅ | ✅ | ✅ |
| KNN | ✅ | ✅ | ❌ | ❌ | ❌ |

### 🔧 When to Use Each Algorithm

**Linear Regression**: When you need to predict continuous values and the relationship is roughly linear.

**Logistic Regression**: When you need classification with probability estimates and interpretable results.

**KNN**: When you have complex, non-linear patterns and don't need to understand the model's decision process.

### 🚀 Next Steps with ML101

1. **Explore More Algorithms**: 
   - Decision Trees and Random Forest
   - Naive Bayes classifiers
   - Support Vector Machines
   - K-Means clustering
   - Principal Component Analysis

2. **Advanced Features**:
   - Cross-validation techniques
   - Hyperparameter tuning
   - Feature selection methods
   - Model ensemble techniques

3. **Real-world Applications**:
   - Apply to your own datasets
   - Compare with scikit-learn
   - Build complete ML pipelines

In [ ]:
print("🎉 Congratulations! You've completed the ML101 tutorial!")
print("\n📚 You've learned about:")
print("   • Linear Regression (Normal Equation & Gradient Descent)")
print("   • Logistic Regression (Binary Classification)")
print("   • K-Nearest Neighbors (Non-linear Classification)")
print("   • Data Preprocessing (Scaling & Feature Engineering)")
print("   • Model Evaluation (Metrics & Visualization)")
print("\n🚀 Next steps:")
print("   • Explore more algorithms: DecisionTree, RandomForest, NaiveBayes, SVM")
print("   • Try the example scripts in examples/")
print("   • Check out the comprehensive docs/ directory")
print("   • Apply these algorithms to real-world datasets")
print("\n📦 ML101 Package:")
print("   • GitHub: https://github.com/yourusername/ML101")
print("   • PyPI: pip install ml101-algorithms")
print("   • Documentation: ./docs/algorithms/README.md")

print("\n✨ Happy Machine Learning! ✨")